# Capstone project week 5

In [33]:
# installing dependencies
!pip install beautifulsoup4
!pip install geopy
!pip install folium

# importing dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium

In [44]:
page = requests.get("https://www.census2011.co.in/states.php")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='alt')
table_rows = table.find_all('tr')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [48]:
import pandas as pd
data = pd.read_html('https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore',skiprows=1)
df = data[1]
df.columns=['State','MAlay','Chinese','Pinyin','Tamil','Region','Area','Population','Density']
df.head()

,State,MAlay,Chinese,Pinyin,Tamil,Region,Area,Population,Density
0,Bedok,*,勿洛,Wù luò,பிடோக்,East,21.69,279380,13000
1,Bishan,NaN,碧山,Bì shān,பீஷான்,Central,7.62,88010,12000
2,Boon Lay,NaN,文礼,Wén lǐ,பூன் லே,West,8.23,30,3.6
3,Bukit Batok,*,武吉巴督,Wǔjí bā dū,புக்கிட் பாத்தோக்,West,11.13,153740,14000
4,Bukit Merah,*,红山,Hóng shān,புக்கிட் மேரா,Central,14.34,151980,11000


In [50]:
select_columns = ['State', 'Region','Area','Population','Density']
df = df[select_columns]
df = df[df.Region == 'Central']
df

,State,Region,Area,Population,Density
1,Bishan,Central,7.62,88010,12000
4,Bukit Merah,Central,14.34,151980,11000
6,Bukit Timah,Central,17.53,77430,4400
12,Downtown Core,Central,4.34,2720,680
13,Geylang,Central,9.64,110200,11400
17,Kallang,Central,9.17,101520,11000
20,Marina East,Central,1.82,*,*
21,Marina South,Central,1.62,*,*
22,Marine Parade,Central,6.12,46390,8000
23,Museum,Central,0.83,420,480


Because the Geocoder service is currently not available and since the number of records is 6, I have queried the longitude and latitude on Google Map. The values will be added to the data frame.

In [51]:
latitudes = [1.323608, 1.2819, 1.3294, 1.2867, 1.3201, 1.3100, 1.2913, 1.2793, 1.3020, 1.2966, 1.3076, 1.3209, 1.3048, 1.2849, 1.2942, 1.2959, 1.3051, 1.2874, 1.2537, 1.2729, 1.3069, 1.3343]
longitudes = [103.9273405, 103.8239,103.8021, 103.8535,103.8918, 103.8651, 103.8709, 103.8702, 103.8971, 103.8485, 103.8404, 103.8424, 103.8318, 103.8439, 103.7861, 103.8361, 103.8509, 103.8460, 103.8257, 103.8627, 103.8189, 103.8563]
df['Latitude'] = pd.Series(latitudes, index=df.index)
df['Longitude'] = pd.Series(longitudes, index=df.index)
df.head()

,State,Region,Area,Population,Density,Latitude,Longitude
1,Bishan,Central,7.62,88010,12000,1.323608,103.92734
4,Bukit Merah,Central,14.34,151980,11000,1.281900,103.82390
6,Bukit Timah,Central,17.53,77430,4400,1.329400,103.80210
12,Downtown Core,Central,4.34,2720,680,1.286700,103.85350
13,Geylang,Central,9.64,110200,11400,1.320100,103.89180


In [52]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [53]:
CLIENT_ID = 'JPIX3XHDO01TA1LSLZZGVPRGZ312ASPMS5FZWBZ3ZGUKJ4OK' # your Foursquare ID
CLIENT_SECRET = '0QPPYFTS14YYUD4AJ4U0H3HKVKAOXFTJBFFGSEJTPMYRI4NX' # your Foursquare Secret
VERSION = '20200419' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius = 500

Your credentails:
CLIENT_ID: JPIX3XHDO01TA1LSLZZGVPRGZ312ASPMS5FZWBZ3ZGUKJ4OK
CLIENT_SECRET:0QPPYFTS14YYUD4AJ4U0H3HKVKAOXFTJBFFGSEJTPMYRI4NX


In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
singapore_venues = getNearbyVenues(names=df['State'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bishan
Bukit Merah
Bukit Timah
Downtown Core
Geylang
Kallang
Marina East
Marina South
Marine Parade
Museum
Newton
Novena
Orchard
Outram
Queenstown
River Valley
Rochor
Singapore River
Southern Islands
Straits View
Tanglin
Toa Payoh


In [57]:
singapore_venues.shape

(874, 7)

In [60]:
singapore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bishan,1.323608,103.92734,Ya Kun Kaya Toast 亞坤,1.324095,103.929198,Coffee Shop
1,Bishan,1.323608,103.92734,FairPrice Fínest,1.324140,103.929260,Supermarket
2,Bishan,1.323608,103.92734,Bedok Chwee Kueh 勿洛水粿,1.324903,103.930250,Chinese Restaurant
3,Bishan,1.323608,103.92734,Haidilao Hot Pot 海底捞火锅,1.324299,103.929104,Hotpot Restaurant
4,Bishan,1.323608,103.92734,Greendot,1.324506,103.930166,Vegetarian / Vegan Restaurant


In [61]:
singapore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bishan,64,64,64,64,64,64
Bukit Merah,10,10,10,10,10,10
Bukit Timah,4,4,4,4,4,4
Downtown Core,63,63,63,63,63,63
Geylang,44,44,44,44,44,44
Kallang,20,20,20,20,20,20
Marina East,4,4,4,4,4,4
Marina South,16,16,16,16,16,16
Marine Parade,18,18,18,18,18,18


In [62]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Neighborhood'] = singapore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bay,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Canal,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Event Space,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,History Museum,Hobby Shop,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Library,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salad Place,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bishan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [64]:
singapore_grouped = singapore_onehot.groupby('Neighborhood').mean().reset_index()
singapore_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bay,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Canal,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Event Space,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,History Museum,Hobby Shop,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Library,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salad Place,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Bishan,0.031250,0.000000,0.0,0.0,0.031250,0.0,0.0,0.0,0.031250,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.015625,0.0,0.015625,0.0,0.000000,0.015625,0.0,0.0,0.000000,0.015625,0.015625,0.0,0.0,0.015625,0.0,0.046875,0.0,0.000000,0.000000,0.062500,0.0,0.015625,0.0,0.000000,0.062500,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.031250,0.0,0.0,0.0,0.0,0.0,0.000000,0.031250,0.000000,0.0,0.015625,0.0,0.046875,0.0,0.0,0.015625,0.03125,0.015625,0.0,0.0,0.015625,0.015625,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.015625,0.0,0.000000,0.000000,0.015625,0.0,0.031250,0.0,0.000000,0.031250,0.0,0.015625,0.0,0.015625,0.0,0.0,0.0,0.03125,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.015625,0.000000,0.031250,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.015625,0.0,0.015625,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.015625,0.0,0.0,0.000000,0.000000,0.0,0.0,0.015625,0.0,0.000000,0.015625,0.0,0.0,0.0,0.0,0.0,0.000000,0.046875,0.046875,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.015625,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.015625,0.000000
1,Bukit Merah,0.000000,0.000000,0.0,0.0,0.100000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0

In [65]:
singapore_grouped.shape

(22, 188)

In [88]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bishan,Chinese Restaurant,Coffee Shop,Sushi Restaurant,Café,Food Court,Supermarket,Indian Restaurant,Asian Restaurant,Bakery,Dessert Shop
1,Bukit Merah,Chinese Restaurant,Convenience Store,Asian Restaurant,Japanese Restaurant,Residential Building (Apartment / Condo),Coffee Shop,Seafood Restaurant,Flea Market,Fried Chicken Joint,French Restaurant
2,Bukit Timah,Pool,Snack Place,Gym / Fitness Center,Gym,Yoga Studio,Filipino Restaurant,French Restaurant,Food Truck,Food Stand,Food Court
3,Downtown Core,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Concert Hall,Yoga Studio,Performing Arts Venue,Hotel Bar,Hotel,Gym / Fitness Center,Salad Place
4,Geylang,Fast Food Restaurant,Chinese Restaurant,Shopping Mall,Asian Restaurant,Bakery,Food Court,Supermarket,Ramen Restaurant,Steakhouse,Coffee Shop


Set the number of clusters to 3 to cluster the neighbourhoods into 3 groups.

In [89]:
# set number of clusters
kclusters = 3

singapore_grouped_clustering = singapore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
      dtype=int32)

In [79]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels2', kmeans.labels_)

In [82]:
singapore_merged = df

# merge singapore_grouped with singapore_data to add latitude/longitude for each neighborhood
singapore_merged = singapore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='State')

singapore_merged.head(15
                    ) # check the last columns!

,State,Region,Area,Population,Density,Latitude,Longitude,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bishan,Central,7.62,88010,12000,1.323608,103.92734,0,0,0,Chinese Restaurant,Coffee Shop,Sushi Restaurant,Café,Food Court,Supermarket,Indian Restaurant,Asian Restaurant,Bakery,Dessert Shop
4,Bukit Merah,Central,14.34,151980,11000,1.281900,103.82390,0,0,0,Chinese Restaurant,Convenience Store,Asian Restaurant,Japanese Restaurant,Residential Building (Apartment / Condo),Coffee Shop,Seafood Restaurant,Flea Market,Fried Chicken Joint,French Restaurant
6,Bukit Timah,Central,17.53,77430,4400,1.329400,103.80210,2,2,2,Pool,Snack Place,Gym / Fitness Center,Gym,Yoga Studio,Filipino Restaurant,French Restaurant,Food Truck,Food Stand,Food Court
12,Downtown Core,Central,4.34,2720,680,1.286700,103.85350,0,0,0,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Concert Hall,Yoga Studio,Performing Arts Venue,Hotel Bar,Hotel,Gym / Fitness Center,Salad Place
13,Geylang,Central,9.64,110200,11400,1.320100,103.89180,0,0,0,Fast Food Restaurant,Chinese Restaurant,Shopping Mall,Asian Restaurant,Bakery,Food Court,Supermarket,Ramen Restaurant,Steakhouse,Coffee Shop
17,Kallang,Central,9.17,101520,11000,1.310000,103.86510,0,0,0,Restaurant,Supermarket,Fast Food Restaurant,BBQ Joint,Coffee Shop,Indian Restaurant,Metro Station,Food Court,Nightclub,Dessert Shop
20,Marina East,Central,1.82,*,*,1.291300,103.87090,1,1,0,History Museum,Golf Course,Park,Golf Driving Range,Yoga Studio,Flea Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand
21,Marina South,Central,1.62,*,*,1.279300,103.87020,0,0,0,Seafood Restaurant,BBQ Joint,Monument / Landmark,Satay Restaurant,Scenic Lookout,Bar,Café,Gift Shop,Food Court,Water Park
22,Marine Parade,Central,6.12,46390,8000,1.302000,103.89710,0,0,0,Chinese Restaurant,Bar,Bakery,Food,Flower Shop,Supermarket,Steakhouse,Karaoke Bar,Food Court,Martial Arts Dojo
23,Museum,Central,0.83,420,480,1.296600,103.84850,0,0,0,Hotel,Japanese Restaurant,Karaoke Bar,Movie Theater,Café,Ice Cream Shop,Bubble Tea Shop,Chinese Restaurant,Dessert Shop,Ramen Restaurant


In [81]:
# create map
latitude = 1.345010
longitude = 103.927340
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['State'], singapore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters